# Reproducibility
- Notebook: tab_platform_overview.ipynb
- Data sources:
  - replication/data/platform-counts.json
  - replication/data/platforms/<platform>/top100.json
- Expected output:
  - Table: Platform-scale overview (top-100 GitHub subset per platform) (tab:platform_overview)


In [1]:
from pathlib import Path
import json
import math
import pandas as pd
import numpy as np


DATA_DIR = (Path.cwd() / ".." / "data").resolve()
PLATFORM_DIR = DATA_DIR / "platforms"

with (DATA_DIR / "platform-counts.json").open("r", encoding="utf-8") as handle:
    platform_counts = json.load(handle)["platforms"]

PLATFORMS = [entry["platform"] for entry in platform_counts]


def load_top100(platform):
    path = PLATFORM_DIR / platform / "top100.json"
    with path.open("r", encoding="utf-8") as handle:
        return json.load(handle)["top100"]


In [2]:
rows = []

for entry in platform_counts:
    platform = entry["platform"]
    top100 = load_top100(platform)

    total_stars = sum((p.get("githubStats") or {}).get("stars", 0) or 0 for p in top100)
    total_downloads = sum(p.get("downloads", 0) or 0 for p in top100)
    total_issues = sum((p.get("githubStats") or {}).get("openIssues", 0) or 0 for p in top100)
    total_forks = sum((p.get("githubStats") or {}).get("forks", 0) or 0 for p in top100)

    rows.append({
        "Platform": entry["display"],
        "Category": entry["category"],
        "All Plugins": entry["all_plugins"],
        "OSS GitHub": entry["oss_github"],
        "Top-100 Size": len(top100),
        "Stars (Top-100)": total_stars,
        "Downloads (Top-100)": total_downloads,
        "Issues (Top-100)": total_issues,
        "Forks (Top-100)": total_forks,
    })

summary = {
    "Platform": "Total",
    "Category": "---",
    "All Plugins": sum(row["All Plugins"] for row in rows),
    "OSS GitHub": sum(row["OSS GitHub"] for row in rows),
    "Top-100 Size": sum(row["Top-100 Size"] for row in rows),
    "Stars (Top-100)": sum(row["Stars (Top-100)"] for row in rows),
    "Downloads (Top-100)": sum(row["Downloads (Top-100)"] for row in rows),
    "Issues (Top-100)": sum(row["Issues (Top-100)"] for row in rows),
    "Forks (Top-100)": sum(row["Forks (Top-100)"] for row in rows),
}

rows.append(summary)

df = pd.DataFrame(rows)

display_df = df.copy()
for col in ["All Plugins", "OSS GitHub", "Top-100 Size", "Stars (Top-100)", "Downloads (Top-100)", "Issues (Top-100)", "Forks (Top-100)"]:
    display_df[col] = display_df[col].map(lambda x: f"{int(x):,}")

display_df


,Platform,Category,All Plugins,OSS GitHub,Top-100 Size,Stars (Top-100),Downloads (Top-100),Issues (Top-100),Forks (Top-100)
0,Chrome,Browser,"246,379","7,459",100,"791,751","233,670,000","33,724","120,128"
1,Firefox,Browser,"110,320","7,862",100,"479,809","17,242,577","12,156","81,394"
2,JetBrains,IDE,"10,003","5,849",100,"501,545","587,304,466","9,213","138,394"
3,VS Code,IDE,"86,145","25,136",100,"184,466","2,950,813,730","32,550","35,166"
4,Sublime,IDE,"5,581","4,694",100,"59,452","49,475,544","1,786","7,912"
5,WordPress,CMS,"59,000","3,986",100,"31,718","7,180,000","11,260","9,336"
6,Minecraft,Gaming,"98,600","26,089",100,"30,385","2,217,620,403","5,332","6,984"
7,Obsidian,Specialized,"2,656","2,656",100,"91,575","38,734,337","8,972","6,932"
8,Home Assistant,Specialized,"5,187","2,389",100,"71,025","4,356,695","5,513","8,487"
9,Total,---,"623,871","86,120",900,"2,241,726","6,106,397,752","120,506","414,733"
